## From the Official KoboldCpp Colab Notebook
You can select a model from the dropdown, or enter a **custom URL** to a GGUF model (Example: `https://huggingface.co/KoboldAI/LLaMA2-13B-Tiefighter-GGUF/resolve/main/LLaMA2-13B-Tiefighter.Q4_K_M.gguf`)

In [ ]:
#@title v-- Enter your model below and then click this to start Koboldcpp
ForceRebuild = False #@param {type:"boolean"}

import os
if not os.path.isfile("/opt/bin/nvidia-smi"):
  raise RuntimeError("⚠️Colab did not give you a GPU due to usage limits, this can take a few hours before they let you back in. Check out https://lite.koboldai.net for a free alternative (that does not provide an API link but can load KoboldAI saves and chat cards) or subscribe to Colab Pro for immediate access.⚠️")

%cd /content
!git clone https://github.com/LostRuins/koboldcpp
%cd /content/koboldcpp
kvers = !(cat koboldcpp.py | grep 'KcppVersion = ' | cut -d '"' -f2)
kvers = kvers[0]
if ForceRebuild:
  kvers = "force_rebuild"
!echo Finding prebuilt binary for {kvers}
!wget -O dlfile.tmp https://kcppcolab.concedo.workers.dev/?{kvers} && mv dlfile.tmp koboldcpp_cublas.so
!test -f koboldcpp_cublas.so && echo Prebuilt Binary Exists || echo Prebuilt Binary Does Not Exist
!test -f koboldcpp_cublas.so && echo Build Skipped || make koboldcpp_cublas LLAMA_CUBLAS=1 LLAMA_COLAB=1 LLAMA_PORTABLE=1
!cp koboldcpp_cublas.so koboldcpp_cublas.dat
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb -q -nv
!apt-get install -qq ./cloudflared-linux-amd64.deb aria2

In [ ]:
#@title Select model (GGUF)
Model = "https://huggingface.co/Lewdiculous/InfinityRP-v1-7B-GGUF-IQ-Imatrix/blob/main/InfinityRP-v1-7B-Q8_0-imat.gguf" # @param ["https://huggingface.co/R136a1/Madang/blob/gguf/lengkuasv2.Q6_K.gguf", "https://huggingface.co/Lewdiculous/Kunoichi-DPO-v2-7B-GGUF-Imatrix/blob/main/Kunoichi-DPO-v2-7B-Q8_0-imatrix.gguf", "https://huggingface.co/Lewdiculous/InfinityRP-v1-7B-GGUF-IQ-Imatrix/blob/main/InfinityRP-v1-7B-Q8_0-imat.gguf", "https://huggingface.co/Lewdiculous/Loyal-Toppy-Bruins-Maid-7B-DARE-GGUF-Imatrix/blob/main/Loyal-Toppy-Bruins-Maid-7B-DARE-Q8_0-imatrix.gguf", "https://huggingface.co/Lewdiculous/BuRP_7B-GGUF-IQ-Imatrix/blob/main/BuRP_7B-Q8_0-imat.gguf", "https://huggingface.co/Epiculous/Fett-uccine-7B-GGUF/blob/main/Fett-uccine-7B-Q8_0.gguf"] {allow-input: true}
Model = Model.replace("blob", "resolve")
name = Model.split("/")[-1]

print("Model list:")
!ls *.gguf
print(f"\nSelected Model: {name}")

In [ ]:
#@title Download model
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M $Model -o $name

In [ ]:
#@title Start Koboldcpp
%cd /content/koboldcpp/
Layers = 99 #@param [99]{allow-input: true}
ContextSize = "8192" # @param ["4096", "6144", "8192"] {allow-input: true}
!python koboldcpp.py $name --usecublas 0 mmq --multiuser --gpulayers $Layers --contextsize $ContextSize --quiet --remotetunnel